# Valores Ausentes

Hoje, modelos de *Machine Learning* não são capazes de identificar dados ausentes, mas se esses dados ainda estiverem no modelo poderá haver falhas ou pode não gerar um resultado tão preciso. Portanto, para rodar um modelo é necessário fazer a limpeza dos dados anteriormente, durante a limpeza serão identificados os dados ausentes. Um dos maiores problemas na hora de fazer a análise exploratória e a limpeza de dados são os dados ausentes. 

A falta de informação pode ser porque um usuário deixou de preencheer um dos dados ou preencheu de forma incorreta, os dados podem ter sido corrompidos, os dados podem não existir de fato, entre outros motivos. 

Quando há dados ausentes uma dúvida sempre reina: excluir os dados, completar eles ou ignorar? Qual a melhor opção? 
Pode-se pensar que a melhor opção é remover os dados que estão ausentes, já que como eles não existem mesmo (em teoria) não irão fazer falta, mas a verdade é que é difícil fornecer uma solução geral, irá depender de cada conjunto de dados. Pode ser que a exclusão dos dados ausentes realmente seja a melhor opção ou pode ser que essa exclusão faça com que na verdade perca dados que são importantes para a análise, então a melhor opção seria completar esses dados seja através da utilização de métodos estatísticos ou indo atrás dos dados que não estão presentes.

O que é recomendado para todos os conjuntos de dados que há valores ausentes é entender o porquê eles estão ausentes, dessa forma ficará fácil saber qual atitude deverá ser tomada em relação a eles. 

Dito isso, nesse *notebook* serão utilizadas algumas técnicas para poder lidar com valores ausentes. 

A base de dados utilizadas é do [Projeto Bike](http://www.maisbikecompartilhada.com.br/).

In [1]:
DATA_PATH = "http://dl.dropboxusercontent.com/s/yyfeoxqw61o3iel/df_rides.csv"

#Importar pacotes necessários
import pandas as pd

#Importar o dataset
df = pd.read_csv(DATA_PATH)

#Ver as primeiras entradas
df.head()

,user_gender,user_birthdate,user_residence,ride_date,time_start,time_end,station_start,station_end,ride_duration,ride_late
0,M,1971-06-08,NaN,2018-01-01,06:05:18,06:21:33,11 - Rodoviária 2,41 - Instituto de Artes,16.250000,0.0
1,M,1989-02-11,DF,2018-01-01,06:27:01,06:32:17,26 - Ministério da Saude,28 - CNMP - Conselho Nacional do Ministério Pú...,5.266667,0.0
2,M,1968-07-19,NaN,2018-01-01,06:29:33,06:44:57,11 - Rodoviária 2,43 - Biblioteca Central,15.400000,0.0
3,M,1991-12-19,NaN,2018-01-01,06:53:53,06:59:45,10 - Ministério dos Transportes,6 - Rodoviária,5.866667,0.0
4,M,1969-03-03,DF,2018-01-01,06:58:56,17:40:04,15 - Brasil 21,11 - Rodoviária 2,641.133333,1.0


## Identificando os Valores Ausentes

Para identificar os valores ausentes será utilizada a função `isnull()` juntamente com a função `sum()`. A soma será útil para que se saiba quantos valores nulos há em cada coluna, como a função `isnull()` gera respostas de True (1) e False(0) a resposta será exatamente uma contagem de valores nulos presentes em cada uma das colunas.

In [2]:
#Ver a quantidade de valores ausentes
df.isnull().sum()

user_gender          396
user_birthdate         1
user_residence    179905
ride_date              0
time_start             0
time_end           43285
station_start          0
station_end            0
ride_duration      73174
ride_late          73174
dtype: int64

In [3]:
#Ver a porcentagem de valores ausentes
print('A porcentagem dos valores ausentes é:')
(df.isnull().sum() / df.shape[0]) * 100

A porcentagem dos valores ausentes é:


user_gender        0.137824
user_birthdate     0.000348
user_residence    62.614419
ride_date          0.000000
time_start         0.000000
time_end          15.064979
station_start      0.000000
station_end        0.000000
ride_duration     25.467594
ride_late         25.467594
dtype: float64

Percebe-se que a coluna `user_residence` possui mais de 62% de valores ausentes. Além dessa coluna há três outras que apresentam uma quantidade significativa de ausência de dados: `time_end`, `ride_duration`, `ride_late`.

## Excluir valores ausentes

Normalmente essa é a primeira opção quando se está lidando com valores nulos no *Data Frame*, mas como dito anteriormente nem sempre essa é a melhor escolha porque ao eliminar os dados de uma única célula eliminará, também, dados existentes em outras colunas. A opção de exclusão de dados deverá ser considerada quando há uma pequena quantidade de dados ausentes a ponto de não ter representatividade no *Data set*. 

A exclusão pode ser feita tanto de linhas com os valores nulos, tanto de colunas inteiras. Para que o *Pandas* entenda qual das duas opções está sendo utilizada é necessário fornecer `axis = 0` quando quer eliminar linhas e `axis = 1` quando o que se quer eliminar são colunas. Essa informação é dada dentro do método `dropna()`.

In [4]:
#Eliminar todas as entradas onde existem valores ausentes em "user_gender"
df_clean = df.dropna(subset=['user_gender'], axis=0)

#Comparar o antes e o depois
print('Antes:\t{}'.format(df.shape))
print('Depois:\t{}'.format(df_clean.shape))

Antes:	(287322, 10)
Depois:	(286926, 10)


## Preencher valores

Normalmente, o preenchimento de valores é a melhor opção pois ela permite que mantenha dados existentes em outras células. 

Quando se opta por preencher dados uma pergunta é muito recorrente: "Com o que será feito o preenchimento desses dados?". A resposta é: depende. 

Há diversas técnicas, como utilizar o valor mais frequente, a média, media, há até técnicas mais avançadas que utilizam modelos de *Machine Learning* para que seja dito qual valor usar nos campos ausentes. A decisão dependerá de cada conjunto de dados e dos recursos disponíveis.

Será utilziada a técnica da mediana para preencher os campos ausentes da coluna `ride_duration`, utilizando a função `fillna()`.

In [5]:
#Antes
df_clean.isnull().sum()

user_gender            0
user_birthdate         1
user_residence    179818
ride_date              0
time_start             0
time_end           43212
station_start          0
station_end            0
ride_duration      73064
ride_late          73064
dtype: int64

In [6]:
#Preencher valores ausentes em 'ride_duration' com a mediana
rd_median = df_clean.ride_duration.median()
df_clean = df_clean.fillna({'ride_duration': rd_median})

#Ver valores ausentes
df_clean.isnull().sum()

user_gender            0
user_birthdate         1
user_residence    179818
ride_date              0
time_start             0
time_end           43212
station_start          0
station_end            0
ride_duration          0
ride_late          73064
dtype: int64

Na coluna `user_gender` há uma variável categória, portanto o ideal nessa coluna é utilizar o valor mais frequente para o preenchimento dos valores.

In [7]:
#Copiar novamente
df_clean = df.copy()

#Ver valores ausentes
df_clean.isnull().sum()

user_gender          396
user_birthdate         1
user_residence    179905
ride_date              0
time_start             0
time_end           43285
station_start          0
station_end            0
ride_duration      73174
ride_late          73174
dtype: int64

In [8]:
#Ver o valor mais frequente
df_clean.user_gender.value_counts()

M    212608
F     74318
Name: user_gender, dtype: int64

In [9]:
#PReencher os valores ausentes de user_gender com 'M'
df_clean = df_clean.fillna({'user_gender': 'M'})

#Ver valores ausentes
df_clean.isnull().sum()

user_gender            0
user_birthdate         1
user_residence    179905
ride_date              0
time_start             0
time_end           43285
station_start          0
station_end            0
ride_duration      73174
ride_late          73174
dtype: int64

## Conclusão 

Nesse *notebook* foram utilizadas algumas técnicas de retirada de valores ausentes. O ponto importante a ser levado em consideração aqui é que o mais ideal é sempre fazer o preenchimento dos dados ausentes ao invés de fazer a remoção dos mesmos.